In [4]:
import pandas as pd

In [5]:
df_historical_data = pd.read_csv('data/fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('data/fifa_worldcup_fixture.csv')
df_missing_data = pd.read_csv('data/fifa_worldcup_missing_data.csv')

## Làm sạch file df_fixture

In [6]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

In [7]:
df_fixture

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


## Làm sạch file df_missing_data và thêm nó vào file df_historical_data

In [8]:
df_missing_data.isnull()
df_missing_data.dropna(inplace=True)
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
419,Brazil,2–0,Costa Rica,2018
420,Serbia,1–2,Switzerland,2018
421,Serbia,0–2,Brazil,2018
408,France,1–0,Peru,2018


In [6]:
df_missing_data

,home,score,away,year
0,Mexico,0–0,Soviet Union,1970
1,Belgium,3–0,El Salvador,1970
2,Soviet Union,4–1,Belgium,1970
3,Mexico,4–0,El Salvador,1970
4,Soviet Union,2–0,El Salvador,1970
...,...,...,...,...
503,Spain,0–1,Switzerland,2010
504,Chile,1–0,Switzerland,2010
505,Spain,2–0,Honduras,2010
506,Chile,1–2,Spain,2010


## Làm sạch file df_historical_data

In [7]:
# Xóa dữ liệu những trận bị xử thua 0-3 do đội không thi đấu
delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden') &
                                  df_historical_data['away'].str.contains('Austria')].index

df_historical_data.drop(index=delete_index, inplace=True)

# căn chỉnh lại khoảng trắng cho các cột score, home, way tương tự ở trên với hàm str strip()
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)
df_historical_data['home'] = df_historical_data['home'].str.strip() 
df_historical_data['away'] = df_historical_data['away'].str.strip()

# tách cột tỷ số thành bàn thắng sân nhà và sân khách và bỏ cột tỷ số
df_historical_data[['HomeGoals', 'AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1, inplace=True)

# Đổi tên các cột, và thay đổi kiểu dữ liệu của chúng
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam', 
                                   'year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# Thêm một cột totalgoals = home goals + away goals
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0,2
420,Serbia,Switzerland,2018,1,2,3
421,Serbia,Brazil,2018,0,2,2
408,France,Peru,2018,1,0,1


# Xuất file được làm sạch với tiền tố clean

In [8]:
df_historical_data.to_csv('data/clean_fifa_worldcup_matches.csv',index=False)
df_fixture.to_csv('data/clean_fifa_worldcup_fixture.csv',index=False)

# Kiểm tra và sắp xếp lại các kì World Cup sao cho ứng với tổng trận đấu

In [9]:
# Xác nhận lại các 
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

for year in years:
    print(year, len(df_historical_data[df_historical_data['Year']==year]))

1930 18
1934 17
1938 18
1950 22
1954 26
1958 35
1962 32
1966 32
1970 32
1974 38
1978 38
1982 52
1986 52
1990 52
1994 52
1998 64
2002 64
2006 64
2010 64
2014 64
2018 64


In [12]:
# Kiểm tra lại file đã làm sạch với trừng hợp xấu nhất là của Sweden
print(df_historical_data[df_historical_data['HomeTeam'].str.contains('Sweden')])
print(df_historical_data[df_historical_data['AwayTeam'].str.contains('Sweden')])

    HomeTeam      AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
23    Sweden     Argentina  1934          3          2           5
46    Sweden          Cuba  1938          8          0           8
66    Sweden         Italy  1950          3          2           5
67    Sweden      Paraguay  1950          2          2           4
74    Sweden         Spain  1950          3          1           4
134   Sweden  West Germany  1958          3          1           4
131   Sweden  Soviet Union  1958          2          0           2
119   Sweden         Wales  1958          0          0           0
118   Sweden       Hungary  1958          2          1           3
115   Sweden        Mexico  1958          3          0           3
466   Sweden        Israel  1970          1          1           2
498   Sweden       Uruguay  1974          3          0           3
494   Sweden      Bulgaria  1974          0          0           0
512   Sweden        Poland  1974          0          1        